In [ ]:
import itertools
import functools
import collections
import operator
import sys
import re
import numpy as np
import math

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2019/"
load_day = functools.partial(load, DIR)

# Problems

## Day 1

http://adventofcode.com/2019/day/1

In [ ]:
content = mapl(int, load_day(1))

__Part 1__

In [ ]:
def get_fuel(mass):
    return max(0, math.floor(mass/3) - 2)

In [ ]:
sum(map(get_fuel, content))

__Part 2__

In [ ]:
def get_over_all_fuel(mass):
    if mass <= 0:
        return 0
    
    fuel = get_fuel(mass)
    return fuel + get_over_all_fuel(fuel)

In [ ]:
sum(map(get_over_all_fuel, content))

## Day 2

http://adventofcode.com/2019/day/2

In [ ]:
content = mapl(int, load_day(2)[0].split(","))

__Part 1__

In [ ]:
def process(program):
    i = 0

    while True:
        if program[i] == 99:
            break

        opcode, idx1, idx2, idx3 = program[i:i+4]

        if opcode == 1:
            program[idx3] = program[idx1] + program[idx2]
        elif opcode == 2:
            program[idx3] = program[idx1] * program[idx2]
        else:
            raise ValueError("Unkown opcode")

        i += 4
        
    return program[0]

In [ ]:
program = content.copy()
program[1] = 12
program[2] = 2
process(program)

__Part 2__

In [ ]:
for i in range(100):
    for j in range(100):
        program = content.copy()
        program[1] = i
        program[2] = j
        if process(program) == 19690720:
            print(100*i+j)
            raise

## Day 3

http://adventofcode.com/2019/day/3

In [ ]:
content = load_day(3)

In [ ]:
path1 = content[0].split(",")
path2 = content[1].split(",")

In [ ]:
directions = {
    "R": (1,0), 
    "L": (-1,0),
    "U": (0,1),
    "D": (0,-1),
}

def get_positions(path):
    positions = list()
    current_position = (0,0)
    for step in path:
        direction, number = step[0], int(step[1:])
        for i in range(number):
            current_position = tuple(add(current_position, directions[direction]))
            positions.append(current_position)
            
    return positions

In [ ]:
positions_path1 = get_positions(path1)
positions_path2 = get_positions(path2)

In [ ]:
commun = set(positions_path1).intersection(set(positions_path2))

__Part 1__

In [ ]:
min([abs(x)+abs(y) for x, y in commun])

__Part 2__

In [ ]:
steps = []
for c in commun:
    idx1 = positions_path1.index(c) + 1
    idx2 = positions_path2.index(c) + 1
    steps.append(idx1 + idx2)

In [ ]:
min(steps)

## Day 4

http://adventofcode.com/2019/day/4

In [ ]:
low = 356261
high = 846303

In [ ]:
def never_decrease(numbers):
    return all(x <= y for x,y in zip(numbers, numbers[1:]))

def contains_a_pair(numbers):
    return any(x == y for x,y in zip(numbers, numbers[1:]))

In [ ]:
pwds = []
for pwd in range(low, high+1):
    numbers = mapl(int, str(pwd))
    if never_decrease(numbers) and contains_a_pair(numbers):
        pwds.append(numbers)

__Part 1__

In [ ]:
len(pwds)

__Part 2__

In [ ]:
sum(1 for pwd in pwds if 2 in collections.Counter(pwd).values())

## Day 5

http://adventofcode.com/2019/day/5

In [ ]:
instructions = mapl(int, load_day(5)[0].split(","))

In [ ]:
def run(instructions, system_id):
    output = None

    def params(ptr, mode):
        return instructions[instructions[ptr]] if mode == 0 else instructions[ptr]

    ptr = 0
    while True:
        opcode = instructions[ptr] % 100
        param1_mode = (instructions[ptr] % 1000) // 100
        param2_mode = (instructions[ptr] % 10000) // 1000

        if opcode == 1:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = a + b
            ptr += 4

        elif opcode == 2:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = a * b
            ptr += 4

        elif opcode == 3:
            instructions[instructions[ptr+1]] = system_id
            ptr += 2

        elif opcode == 4:
            output = params(ptr+1, param1_mode)
            ptr += 2

        elif opcode == 5:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            ptr = b if a != 0 else ptr+3

        elif opcode == 6:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            ptr = b if a == 0 else ptr+3

        elif opcode == 7:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = int(a < b)
            ptr += 4

        elif opcode == 8:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = int(a == b)
            ptr += 4

        elif opcode == 99:
            break

        else:
            print(opcode)
            raise
            
    return output

__Part 1__

In [ ]:
run(instructions.copy(), 1)

__Part 2__

In [ ]:
run(instructions.copy(), 5)

## Day 6

http://adventofcode.com/2019/day/6

In [ ]:
content = mapl(lambda x: x.split(")"), load_day(6))

In [ ]:
parents = dict()
childrens = collections.defaultdict(set)

for u,v in content:
    parents[v] = u
    childrens[u].add(v)

__Part 1__

In [ ]:
total = 0
for obj in parents.keys():
    while obj != "COM":
        total += 1
        obj = parents[obj]
        
total

__Part 2__

In [ ]:
start = "YOU"
end = "SAN"

explored = set()

def explore(source, root, transfers):
    explored.add(root)
    
    if root == "SAN":
        return transfers
    
    nodes = childrens[root]
    if root in parents: 
        nodes.add(parents[root])
    
    for node in nodes:
        if node not in explored:
            out = explore(root, node, transfers+1)
            if out is not None:
                return out

In [ ]:
explore(start, parents[start], 0) - 1 

## Day 7

http://adventofcode.com/2019/day/7

In [ ]:
content = load_day(7)

__Part 1__

__Part 2__

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)

__Part 1__

__Part 2__